# NHiTS Trade Forecasting - Colab Setup

이 노트북은 Google Colab에서 NHiTS 예측 파이프라인을 실행하기 위한 설정입니다.


## 1. 저장소 클론 및 디렉토리 구조 생성


In [ ]:
# 저장소 클론 (GitHub에 업로드한 경우)
# !git clone <your-repo-url>
# %cd <repo-name>

# 또는 직접 파일 업로드 후 디렉토리 구조 생성
import os
from pathlib import Path

# 작업 디렉토리 설정
WORK_DIR = Path("/content/NHiTS")
WORK_DIR.mkdir(exist_ok=True)
%cd {WORK_DIR}

# 필요한 디렉토리 생성
os.makedirs("data", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("results", exist_ok=True)

print(f"작업 디렉토리: {WORK_DIR}")


## 2. 데이터 파일 업로드

아래 셀을 실행하여 데이터 파일을 업로드하세요.

필요한 파일:
- 8개 타겟 CSV: China_Export.csv, China_Import.csv, Korea_Export.csv, Korea_Import.csv, Taiwan_Export.csv, Taiwan_Import.csv, World_Export.csv, World_Import.csv
- 외생변수 CSV: tariff.csv, USD_KRW.csv, USD_CNY.csv, USD_TWD.csv, WTI.csv, copper.csv


In [ ]:
from google.colab import files
import shutil

# 파일 업로드 (여러 파일 선택 가능)
uploaded = files.upload()

# 업로드된 파일을 data/ 폴더로 이동
for filename in uploaded.keys():
    if filename.endswith('.csv'):
        shutil.move(filename, f"data/{filename}")
        print(f"Moved {filename} to data/")
    else:
        print(f"Skipped {filename} (not a CSV file)")


## 5. 파이프라인 실행 (Import 방식 - tqdm 출력 개선)


In [ ]:
# tqdm 출력 형식 개선 (Colab 환경에 최적화)
import sys
from IPython.display import clear_output
from tqdm import tqdm
import tqdm.notebook as tqdm_notebook

# tqdm 설정 (Colab에서 더 깔끔하게 보이도록)
tqdm.pandas = tqdm_notebook.tqdm.pandas

# 또는 tqdm을 notebook 모드로 설정
try:
    from tqdm.notebook import tqdm
    print("Using tqdm notebook mode for better Colab display")
except:
    from tqdm import tqdm
    print("Using standard tqdm")

# 파이프라인 실행 (import 방식)
from src.pipeline.run_all import main as run_pipeline

print("="*60)
print("Starting NHiTS Pipeline...")
print("="*60)

# 전체 파이프라인 실행
run_pipeline()


또는 개별 실행:


In [ ]:
# 개별 실행 (선택적)

# 학습만
# from src.pipeline.train import main as train_main
# train_main()

# 2024년 검증만
# from src.pipeline.evaluate import main_2024
# main_2024()

# 예측만
# from src.pipeline.forecast import main as forecast_main
# forecast_main()


## 3. 패키지 설치


In [ ]:
# requirements.txt가 있다면
!pip install -r requirements.txt

# 또는 직접 설치
# !pip install neuralforecast torch pandas numpy scikit-learn matplotlib pydantic tqdm


## 4. GPU 확인 (선택사항)


In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")


## 5. 파이프라인 실행


## 7. 2024년 검증 결과 시각화 (전체 기간)


In [ ]:
# 2024년 검증 결과를 전체 기간과 함께 시각화
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from src.data import load_target_df
from src.config import ExogenousConfig, RAW_DATA_DIR

# 결과 디렉토리
results_dir = Path("results")
results_dir.mkdir(exist_ok=True)

# 타겟 목록
targets = [
    "China_Export", "China_Import",
    "Korea_Export", "Korea_Import",
    "Taiwan_Export", "Taiwan_Import",
    "World_Export", "World_Import",
]

exog_config = ExogenousConfig()

# 먼저 2024년 검증 파일이 있는지 확인
validation_files_exist = []
for target in targets:
    validation_file = results_dir / f"{target}_2024_validation.csv"
    if validation_file.exists():
        validation_files_exist.append(target)

if len(validation_files_exist) == 0:
    print("="*60)
    print("⚠️  2024년 검증 결과 파일이 없습니다!")
    print("="*60)
    print("\n먼저 2024년 검증을 실행해주세요:")
    print("\n  from src.pipeline.evaluate import main_2024")
    print("  main_2024()")
    print("\n또는:")
    print("\n  !python -m src.pipeline.evaluate 2024")
    print("\n" + "="*60)
else:
    print(f"✓ {len(validation_files_exist)}개 타겟의 2024년 검증 결과를 찾았습니다.")
    print(f"  타겟: {', '.join(validation_files_exist)}")
    print()

# 각 타겟에 대해 시각화
for target in targets:
    try:
        # 1. 2024년 검증 결과 로드
        validation_file = results_dir / f"{target}_2024_validation.csv"
        if not validation_file.exists():
            print(f"⚠️  {validation_file.name} not found. Skipping {target}.")
            continue
        
        val_df = pd.read_csv(validation_file)
        val_df["ds"] = pd.to_datetime(val_df["ds"])
        val_df = val_df.sort_values("ds").reset_index(drop=True)
        
        # 2. 전체 기간 데이터 로드
        try:
            full_df = load_target_df(target, RAW_DATA_DIR, exog_config)
            full_df["ds"] = pd.to_datetime(full_df["ds"])
            full_df = full_df.sort_values("ds").reset_index(drop=True)
        except Exception as e:
            print(f"Warning: Could not load full data for {target}: {e}")
            full_df = None
        
        # 3. 그래프 생성
        plt.figure(figsize=(14, 6))
        
        if full_df is not None:
            # 전체 기간의 실제값 표시
            train_df = full_df[full_df["ds"] <= "2023-12-01"]
            val_actual_df = full_df[full_df["ds"] > "2023-12-01"]
            
            # 학습 기간 실제값 (회색)
            plt.plot(train_df["ds"], train_df["y"], 
                    color="gray", alpha=0.6, linewidth=1.5, 
                    label="Training Data (Actual)", linestyle="-")
            
            # 검증 기간 실제값 (검은색)
            if len(val_actual_df) > 0:
                plt.plot(val_actual_df["ds"], val_actual_df["y"], 
                        color="black", linewidth=2.5, marker="o", markersize=6,
                        label="2024 Actual", linestyle="-")
        else:
            # 전체 데이터가 없으면 검증 데이터의 실제값만 표시
            plt.plot(val_df["ds"], val_df["y_actual"], 
                    color="black", linewidth=2.5, marker="o", markersize=6,
                    label="2024 Actual", linestyle="-")
        
        # 95% 신뢰구간 표시 (있는 경우)
        if "y_pred_lower_95" in val_df.columns and "y_pred_upper_95" in val_df.columns:
            plt.fill_between(
                val_df["ds"], 
                val_df["y_pred_lower_95"], 
                val_df["y_pred_upper_95"],
                color="gold", alpha=0.3, 
                label="95% Confidence Interval"
            )
        
        # 검증 기간 예측값 (파란색)
        plt.plot(val_df["ds"], val_df["y_pred"], 
                color="blue", linewidth=2, marker="s", markersize=6, alpha=0.8,
                label="2024 Forecast", linestyle="--")
        
        # 2023-12-01에 수직선 추가 (학습/검증 구분)
        plt.axvline(x=pd.to_datetime("2023-12-01"), 
                   color="red", linestyle=":", linewidth=2, alpha=0.7,
                   label="Train/Val Split")
        
        # 그래프 설정
        plt.title(f"{target} - Full Period with 2024 Validation", 
                 fontsize=14, fontweight="bold")
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Trade Balance", fontsize=12)
        plt.legend(loc="best", fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        # 저장
        output_file = results_dir / f"{target}_full_period_validation.png"
        plt.savefig(output_file, dpi=150, bbox_inches="tight")
        print(f"Saved: {output_file}")
        
        # 표시
        plt.show()
        
    except Exception as e:
        print(f"Error processing {target}: {e}")
        import traceback
        traceback.print_exc()
        continue

print("\nVisualization complete!")


## 8. 2024년 검증 실행 (파일이 없을 경우)


In [ ]:
# 2024년 검증 실행 (검증 파일이 없을 경우)
from tqdm.notebook import tqdm
from src.pipeline.evaluate import main_2024

print("="*60)
print("2024년 검증 실행 중...")
print("="*60)

# 2024년 검증 실행
metrics = main_2024()

print("\n✓ 2024년 검증 완료!")
print("이제 위의 시각화 셀을 다시 실행하세요.")


In [ ]:
# 전체 파이프라인 실행 (학습 + 평가 + 예측)
!python -m src.pipeline.run_all


또는 개별 실행:


In [ ]:
# 학습만
# !python -m src.pipeline.train

# 평가만
# !python -m src.pipeline.evaluate

# 예측만
# !python -m src.pipeline.forecast


## 6. 결과 다운로드


In [ ]:
from google.colab import files
import zipfile

# results 폴더를 zip으로 압축
with zipfile.ZipFile('results.zip', 'w') as zipf:
    for root, dirs, files_list in os.walk('results'):
        for file in files_list:
            zipf.write(os.path.join(root, file))

# 다운로드
files.download('results.zip')

print("results.zip 다운로드 완료!")
